In [1]:
using Flux , Distributions , DifferentialEquations
using Base.Iterators: repeated
using Flux.Data.MNIST

┌ Info: CUDAdrv.jl failed to initialize, GPU functionality unavailable (set JULIA_CUDA_SILENT or JULIA_CUDA_VERBOSE to silence or expand this message)
└ @ CUDAdrv /Users/bb/.julia/packages/CUDAdrv/b1mvw/src/CUDAdrv.jl:67


In [3]:
function phi(x)
    return sum(x.^2, dims=1)
end        

phi (generic function with 1 method)

In [4]:
d = 10
T = 1.0
num_samples = 1000

1000

In [5]:
xi = rand(Uniform(0. , 1.) , d , num_samples)

10×1000 Array{Float64,2}:
 0.383458  0.948418   0.247086   0.796964  …  0.398945  0.643064  0.916923 
 0.534082  0.377529   0.407166   0.591351     0.277371  0.881913  0.627506 
 0.956493  0.130554   0.0151838  0.307506     0.994351  0.446721  0.664145 
 0.694612  0.0065483  0.803769   0.163412     0.697235  0.77661   0.0346157
 0.962779  0.868907   0.586146   0.739943     0.718693  0.892328  0.506592 
 0.976926  0.151219   0.148845   0.391477  …  0.582533  0.469572  0.0569227
 0.639683  0.327828   0.723714   0.259486     0.967146  0.657627  0.986668 
 0.958816  0.403926   0.467515   0.457937     0.658228  0.934489  0.162377 
 0.947779  0.420736   0.610676   0.145043     0.712135  0.643412  0.649774 
 0.718469  0.9469     0.994857   0.366016     0.999084  0.730869  0.417754 

In [6]:
N = Normal(0 , sqrt(2*T))

Normal{Float64}(μ=0.0, σ=1.4142135623730951)

In [7]:
x_sde = xi + rand(N , d , num_samples)

10×1000 Array{Float64,2}:
  0.756758     1.58237    -0.895761  …   1.7693     -2.86815    0.742072
 -1.28205     -0.430521    1.05164       0.792912    4.41202    0.345062
  0.563201    -2.13748    -2.93591      -0.186056    1.34653   -0.196038
  0.407215     2.20508     0.531844      2.52158     0.51678   -0.99932 
  2.01086      1.65753     1.65626       3.71559    -0.112249   2.63943 
 -0.43166      0.336837    1.04138   …   0.0910626  -1.92788   -1.45193 
 -1.80723      0.0654994  -0.368803      0.208969    1.2116     1.35346 
  0.779907     1.05132     0.901446      2.50425     0.544318   0.786075
  1.83633     -0.749998    0.615953      0.275813   -0.429606  -0.769445
 -0.00637268   1.31991     1.9608        0.435173   -3.00777    0.61558 

In [8]:
y = phi(x_sde)

1×1000 Array{Float64,2}:
 14.1758  18.3955  19.8112  36.9394  …  21.752  30.5465  44.4972  14.2022

In [9]:
chain = Flux.Chain(Dense(10,128,tanh),Dense(128,256 ,tanh ) , Dense(256,256 ,tanh) , Dense(256,128 ,tanh) , Dense(128 , 1) )

Chain(Dense(10, 128, tanh), Dense(128, 256, tanh), Dense(256, 256, tanh), Dense(256, 128, tanh), Dense(128, 1))

In [15]:
loss(x , y) =Flux.mse(chain(x), y)
function evalcbw()
    @show loss(x_sde , y)
end

evalcbw (generic function with 1 method)

In [16]:
dataset = repeated((x_sde, y), 1000)
max_err = 10000
opt = ADAM()
Flux.train!(loss, Flux.params(chain),dataset, opt, cb = evalcbw)

loss(x_sde, y) = 499.8660351067217
loss(x_sde, y) = 452.79884663536166
loss(x_sde, y) = 410.8481486642088
loss(x_sde, y) = 376.05022522068333
loss(x_sde, y) = 348.91070007257156
loss(x_sde, y) = 328.67086769344104
loss(x_sde, y) = 314.01901175233473
loss(x_sde, y) = 303.56606946006053
loss(x_sde, y) = 296.0272775601717
loss(x_sde, y) = 290.301940074372
loss(x_sde, y) = 285.5565351725442
loss(x_sde, y) = 281.31532379474777
loss(x_sde, y) = 277.46763957998144
loss(x_sde, y) = 274.05541669398514
loss(x_sde, y) = 270.9825083751451
loss(x_sde, y) = 268.00623169141716
loss(x_sde, y) = 264.930952635702
loss(x_sde, y) = 261.6901058704401
loss(x_sde, y) = 258.2786048370626
loss(x_sde, y) = 254.6671522635918
loss(x_sde, y) = 250.8005239970139
loss(x_sde, y) = 246.65789505199612
loss(x_sde, y) = 242.295858433068
loss(x_sde, y) = 237.8445886894246
loss(x_sde, y) = 233.4405379534794
loss(x_sde, y) = 229.11484171947987
loss(x_sde, y) = 224.76234566964246
loss(x_sde, y) = 220.2383910938496
loss(x_sde

InterruptException: InterruptException:

In [17]:
chain(x_sde)

1×1000 Array{Float32,2}:
 23.5582  23.5599  23.5533  23.5614  …  23.5473  23.5615  23.518  23.5575

In [18]:
y

1×1000 Array{Float64,2}:
 14.1758  18.3955  19.8112  36.9394  …  21.752  30.5465  44.4972  14.2022

1×1000 Array{Float64,2}:
 16.0786  7.67469  9.98907  30.2885  …  20.2667  16.2465  33.5881  32.3608